<a href="https://colab.research.google.com/github/btg1631/study_data_analytics/blob/main/docs/quests/CDAs/CDA_mixed_unVar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## load dataset file

In [36]:
import pandas as pd
df_fruits = pd.read_csv('fruitstore_saleshistory.csv')
df_fruits[:2]

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977,매장C,NaN,10:05


In [37]:
df_fruits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705571 entries, 0 to 705570
Data columns (total 20 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   공급일자      705571 non-null  object 
 1   요일        705571 non-null  object 
 2   공급월       705571 non-null  int64  
 3   공급주차      705571 non-null  int64  
 4   회원번호      705571 non-null  int64  
 5   조합원상태     705571 non-null  object 
 6   물품대분류     705571 non-null  object 
 7   물품중분류     705571 non-null  object 
 8   물품소분류     705571 non-null  object 
 9   물품명       705571 non-null  object 
 10  구매수량      705571 non-null  float64
 11  주소-구      705571 non-null  object 
 12  주소-동      705571 non-null  object 
 13  성별        705566 non-null  object 
 14  연령        705538 non-null  float64
 15  연령대       705538 non-null  object 
 16  구매금액      705571 non-null  int64  
 17  구매매장      705571 non-null  object 
 18  반품_원거래일자  11234 non-null   float64
 19  구매시각      705571 non-null  object 
dtypes: f

In [38]:
# 결측치 확인
df_fruits.isnull().sum()

공급일자             0
요일               0
공급월              0
공급주차             0
회원번호             0
조합원상태            0
물품대분류            0
물품중분류            0
물품소분류            0
물품명              0
구매수량             0
주소-구             0
주소-동             0
성별               5
연령              33
연령대             33
구매금액             0
구매매장             0
반품_원거래일자    694337
구매시각             0
dtype: int64

## 범주형 / 연속형이고 등분산인 경우

- 성별(범주형)에 따라 구매한 상품의 단가(연속형) 분산이 같은지 확인합니다.
   + 성별을 기준으로 상품 단가의 등분산성 여부를 검정하여, 성별에 따른 구매 패턴의 차이를 분석합니다


### 분석 내용
- 귀무 가설 : 성별에 따른 구매 패턴에 차이가 없다.
- 대립 가설 : 성별에 따른 구매 패턴에 차이가 있다.

In [39]:
import scipy.stats as stats

# 구매금액(연속형) 정규성 검증  -->  비정규 분포
result = stats.anderson(df_fruits['구매금액'].dropna())
print('Test Statistic: %.3f' % result.statistic)
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    if result.statistic < cv:
        print('%.1f%% 수준에서 데이터는 정규 분포를 따름 (검정 통계량 %.3f < 임계값 %.3f)' % (sl, result.statistic, cv))
    else:
        print('%.1f%% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 %.3f >= 임계값 %.3f)' % (sl, result.statistic, cv))

Test Statistic: 80887.237
15.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.576)
10.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.656)
5.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.787)
2.5% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.918)
1.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 1.092)


In [40]:
# 성별(항목 2개) --> Wilcoxon Test
df_fruits['성별'].value_counts()

여    662024
남     43542
Name: 성별, dtype: int64

In [41]:
conditions = '성별 == "여"'
series_women_price = df_fruits.query(conditions)['구매금액']
series_women_price

0         22207
1          4977
2          7083
3           766
4          4403
          ...  
705566     9189
705567     5169
705568     7466
705569     3542
705570     1914
Name: 구매금액, Length: 662024, dtype: int64

In [42]:
conditions = '성별 == "남"'
series_men_price = df_fruits.query(conditions)['구매금액']
series_men_price

30        16273
34         3829
38        20676
71         7083
90        16081
          ...  
705445     4020
705446     9189
705481     2967
705502    49775
705542     7466
Name: 구매금액, Length: 43542, dtype: int64

In [43]:
# Wilcoxon Test
stats.ranksums(series_women_price, series_men_price)

RanksumsResult(statistic=-1.571683301008152, pvalue=0.11602401682141163)

#### 분석 결과
- 통계 결론 : 귀무가설 성립 (statistic=-1.57, pvalue=0.11)
- 사실 결론 : 성별에 따른 구매 패턴에 차이가 없다.